In [13]:
import cv2
import os
import rosbag2_py
import datetime
import json
from io import BytesIO

from matplotlib import pyplot as plt
from matplotlib.figure import Figure
from matplotlib import gridspec
import pandas as pd
import numpy as np

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

from tqdm.notebook import tqdm, trange

from cv_bridge import CvBridge
bridge = CvBridge()

from sensor_msgs.msg import Image
from deepracer_interfaces_pkg.msg import InferResultsArray, InferResults
from rclpy.serialization import deserialize_message

from deepracer_viz.gradcam.cam import GradCam
from deepracer_viz.model.model import Model
from deepracer_viz.model.metadata import ModelMetadata


In [14]:
def get_rosbag_options(path, serialization_format='cdr'):
    storage_options = rosbag2_py.StorageOptions(uri=path, storage_id='sqlite3')

    converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format=serialization_format,
        output_serialization_format=serialization_format)

    return storage_options, converter_options

In [15]:
def create_plot(action_names, height, width, dpi):
   
    fig = plt.figure(figsize=(width/dpi,height/dpi), dpi=dpi)
    x = list(range(0,len(action_names)))

    spec = gridspec.GridSpec(ncols=3, nrows=2,
                            width_ratios=[1, 1, 1], wspace=0.1,
                            hspace=0.1, height_ratios=[3, 1], left=0.05,right=0.95,top=0.99,bottom=0.2)
    ax00 = fig.add_subplot(spec[0])
    plt.setp(ax00, xlabel="Original")
    plt.tick_params(labelleft = False, labelbottom=False)

    ax01 = fig.add_subplot(spec[1])
    plt.setp(ax01, xlabel="GradCam")
    plt.tick_params(labelleft = False, labelbottom=False)

    ax02 = fig.add_subplot(spec[2])
    plt.setp(ax02, xlabel="GradCam-Edit")
    plt.tick_params(labelleft = False, labelbottom=False)

    ax10 = fig.add_subplot(spec[3])
    ax10.set_ylim(0.0, 1.0)
    plt.setp(ax10, xlabel="OpenVINO")
    plt.xticks(x,action_names[::-1],rotation='vertical')

    ax11 = fig.add_subplot(spec[4], sharey=ax10)
    ax11.set_ylim(0.0, 1.0)
    plt.setp(ax11, xlabel="Tensorflow")
    plt.xticks(x,action_names[::-1],rotation='vertical')

    ax12 = fig.add_subplot(spec[5], sharey=ax10)
    ax12.set_ylim(0.0, 1.0)
    plt.setp(ax12, xlabel="Tensorflow-Edit")
    plt.xticks(x,action_names[::-1],rotation='vertical')

    fig.canvas.draw()

    return fig

In [16]:
def update_plot(fig: Figure, step, action_names, img, gradcam, gradcam2):

    x = list(range(0,len(action_names)))

    openvino_result = pd.DataFrame(step['openvino_results'])
    tf_result = pd.DataFrame(step['tf_results'])
    tf_result_2 = pd.DataFrame(step['tf_results_2'])

    ax = fig.get_axes()

    for a in ax:
        for p in set(a.containers):
            p.remove()
        for i in set(a.images):
            i.remove()

    ax[0].imshow(img)
    ax[1].imshow(gradcam)
    ax[2].imshow(gradcam2)
    ax[3].bar(x,openvino_result['probability'][::-1], color='blue')
    ax[4].bar(x,tf_result['probability'][::-1], color='blue')
    ax[5].bar(x,tf_result_2['probability'][::-1], color='blue')
    fig.canvas.draw()

    buf = fig.canvas.buffer_rgba()
    ncols, nrows = fig.canvas.get_width_height()
    return cv2.cvtColor(np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 4), cv2.COLOR_RGBA2BGR)

In [17]:
def get_gradient_values(gradient_img, multiplier=1):
    """ Given the image gradient returns gradient_alpha_rgb_mul and one_minus_gradient_alpha.
    These pre-calculated numbers are used to apply the gradient on the camera image

    Arguments:
        gradient_img (Image): Gradient image that has to applied on the camera image
        multiplier (float): This decides what percentage of gradient images alpha has to be applied.
                            This is useful in fading feature.

    Returns:
        (tuple): gradient_alpha_rgb_mul (Numpy.Array) gradient_img * gradient_alpha value
                 one_minus_gradient_alpha (Numpy.Array) (1 - gradient_alpha)
    """
    (height, width, _) = gradient_img.shape
    gradient_alpha = (gradient_img[:, :, 3] / 255.0 * multiplier).reshape(height, width, 1)

    gradient_alpha_rgb_mul = gradient_img * gradient_alpha
    one_minus_gradient_alpha = (1 - gradient_alpha).reshape(height, width)
    return gradient_alpha_rgb_mul, one_minus_gradient_alpha

In [18]:
def apply_gradient(main_image, gradient_alpha_rgb_mul, one_minus_gradient_alpha):
    """ The gradient on the image is overlayed so that text looks visible and clear.
    This leaves a good effect on the image.
    The older code took 6.348s for 1000 runs

    Numpy broadcasting is slower than normal python
    major_cv_image_1[:, :, :4] = (gradient_alpha_rgb_mul + (major_cv_image_1 * one_minus_gradient_alpha))[:, :, :4]
    Timeit 1000 runs - 6.523s

    The current code takes - 5.131s for 1000 runs

    Args:
        main_image (Image): The main image where gradient has to be applied
        gradient_alpha_rgb_mul (Numpy.Array): gradient_img * gradient_alpha value
        one_minus_gradient_alpha (Numpy.Array): (1 - gradient_alpha)
    Returns:
        Image: Gradient applied image
    """
    for channel in range(0, 4):
        main_image[:, :, channel] = gradient_alpha_rgb_mul[:, :, channel] + \
            (main_image[:, :, channel] * one_minus_gradient_alpha)
    return main_image

In [19]:
def get_image(image_path, img_size=None):
    """ Given the icon_name in the track_iconography folder without png, gives back cv2 image
    with all 4 channels
    Args:
        icon_name (str): The name of the icon in the track_iconography folder
        img_size (tuple): If you want to resize the image (width, height) (default: {None})
    Returns:
        Image: The cv2 image read from the .png file
    """
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)
    if img_size:
        image = cv2.resize(image, img_size)
    return image


In [20]:
bag_path = '/workspace/logs/deepracer-bag-20221201-155319'
metadata_json = '/workspace/logs/model/model_metadata.json'
model_pb = '/workspace/logs/model/model.pb'
overlay_img = '/workspace/analysis/resources/fade_gray_overlay.png'
CODEC = "avc1"
output_file = "/workspace/logs/test.mp4"
width = 1350
height = 600

In [21]:
gradient_img = get_image(overlay_img)
gradient_img = cv2.cvtColor(gradient_img, cv2.COLOR_RGBA2BGRA)
gradient_alpha_rgb_mul, one_minus_gradient_alpha = get_gradient_values(gradient_img)

In [22]:
storage_options, converter_options = get_rosbag_options(bag_path)

reader = rosbag2_py.SequentialReader()
reader.open(storage_options, converter_options)
storage_filter = rosbag2_py.StorageFilter(topics=['/inference_pkg/rl_results'])

metadata = ModelMetadata.from_file(metadata_json)
model = Model.from_file(model_pb_path=model_pb, metadata=metadata)

Device mapping: no known devices.


[INFO] [1675198180.606949726] [rosbag2_storage]: Opened database '/workspace/logs/deepracer-bag-20221201-155319/deepracer-bag-20221201-155319_0.db3' for READ_ONLY.


In [23]:
reader.set_filter(storage_filter)

first_stamp = -1
steps_data = {'steps': []}

s = 0

while reader.has_next() and s < 15:
    step = {}
    
    (topic, data, t) = reader.read_next()
    msg = deserialize_message(data, InferResultsArray)

    # Timestamp
    timestamp: float = (msg.images[0].header.stamp.sec + msg.images[0].header.stamp.nanosec / 1e9)

    if first_stamp == -1:
        first_stamp = timestamp
        timestamp = 0
    else:
        timestamp = timestamp - first_stamp

    step['timestamp'] = timestamp
    step['seq'] = int(msg.images[0].header.frame_id)

    s += 1
    steps_data['steps'].append(step)

while reader.has_next():
    (topic, data, t) = reader.read_next()
    s += 1
    
df = pd.json_normalize(steps_data['steps'])
del steps_data

step_diff = df['seq'].max() - df['seq'].min()
fps = step_diff / df['timestamp'].max()
tmp_img = bridge.compressed_imgmsg_to_cv2(msg.images[0], desired_encoding="passthrough")

print("Start time: {}".format(datetime.datetime.fromtimestamp(first_stamp)))
print("Loaded {} steps from {}.".format(len(df.index), step_diff + 1))
print("Duration: {:.2f} seconds".format(df['timestamp'].max()))
print("Average FPS: {:.1f}".format(fps))
print("Action Space: {} actions".format(len(msg.results)))
print("Input image: {}x{}, {} channels.".format(tmp_img.shape[1], tmp_img.shape[0], tmp_img.shape[2]))
print("Total messages: {}".format(s))

Start time: 2022-12-01 20:53:19.335952
Loaded 15 steps from 15.
Duration: 0.47 seconds
Average FPS: 30.0
Action Space: 11 actions
Input image: 640x480, 3 channels.
Total messages: 4521


In [ ]:
reader = rosbag2_py.SequentialReader()
reader.open(storage_options, converter_options)
reader.set_filter(storage_filter)

first_stamp = -1
steps_data = {'steps': []}

action_names = []
with open(metadata_json,"r") as jsonin:
    model_metadata=json.load(jsonin)
for action in model_metadata['action_space']:
    action_names.append(str(action['steering_angle'])+ u'\N{DEGREE SIGN}' + " "+"%.1f"%action["speed"])

writer = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*CODEC), fps, (width, height))
fig = create_plot(action_names, height, width, 72)

p_bar = tqdm(total=s)

s = 0
with model.session as sess:

    cam = GradCam(model, model.get_conv_outputs())

    while reader.has_next():
        step = {}
        
        (topic, data, t) = reader.read_next()
        msg = deserialize_message(data, InferResultsArray)

        # Timestamp
        timestamp: float = (msg.images[0].header.stamp.sec + msg.images[0].header.stamp.nanosec / 1e9)

        if first_stamp == -1:
            first_stamp = timestamp
            timestamp = 0
        else:
            timestamp = timestamp - first_stamp

        step['timestamp'] = timestamp
        step['seq'] = int(msg.images[0].header.frame_id)

        # Extract original image from first camera
        cv_img_bgra = bridge.compressed_imgmsg_to_cv2(msg.images[0], desired_encoding="passthrough")
        cv_img = cv2.cvtColor(cv_img_bgra, cv2.COLOR_BGRA2RGB)
        # step['img'] = cv_img
        
        # Find best OpenVINO Result
        step['openvino_action'] = {'action': -1, 'probability': -1}    
        step['openvino_results'] = []
        for r in msg.results:
            step['openvino_results'].append({'action': r.class_label, 'probability': r.class_prob})
            if r.class_prob > step['openvino_action']['probability']:
                step['openvino_action'] = {'action': r.class_label, 'probability': r.class_prob}

        # Process image with Tensorflow
        tf_result, grad_img = cam.process(cv_img)
        #step['gradcam_img'] = grad_img
        
        step['tf_action'] = {'action': -1, 'probability': -1}    
        step['tf_results'] = []
        for i, r in enumerate(tf_result):
            step['tf_results'].append({'action': i, 'probability': r})
            if r > step['tf_action']['probability']:
                step['tf_action'] = {'action': i, 'probability': r}


        # Edit image and process with Tensorflow
        ed_img = cv2.cvtColor(cv_img_bgra, cv2.COLOR_RGB2RGBA)
        ed_img = apply_gradient(ed_img, gradient_alpha_rgb_mul, one_minus_gradient_alpha)
        ed_img = cv2.cvtColor(ed_img, cv2.COLOR_BGRA2RGB)
        tf_result_2, grad_img_2 = cam.process(ed_img)
        #step['gradcam_img'] = grad_img
        
        step['tf_action_2'] = {'action': -1, 'probability': -1}    
        step['tf_results_2'] = []
        for i, r in enumerate(tf_result_2):
            step['tf_results_2'].append({'action': i, 'probability': r})
            if r > step['tf_action_2']['probability']:
                step['tf_action_2'] = {'action': i, 'probability': r}

        # Results
        step['results'] = []

        # Plot image
        img = update_plot(fig, step, action_names, cv_img, grad_img, grad_img_2)
        
        p_bar.update(1)

        steps_data['steps'].append(step)

        writer.write(img)


writer.release()
plt.close(fig)
p_bar.close()

[INFO] [1675198181.976651466] [rosbag2_storage]: Opened database '/workspace/logs/deepracer-bag-20221201-155319/deepracer-bag-20221201-155319_0.db3' for READ_ONLY.


  0%|          | 0/4521 [00:00<?, ?it/s]

localhost/replica:0/task:0/device:CPU:0
one_hot/indices: (Const): /job:localhost/replica:0/task:0/device:CPU:0
one_hot/depth: (Const): /job:localhost/replica:0/task:0/device:CPU:0
Sum/reduction_indices: (Const): /job:localhost/replica:0/task:0/device:CPU:0
gradients/grad_ys_0/Const: (Const): /job:localhost/replica:0/task:0/device:CPU:0
gradients/Sum_grad/Size: (Const): /job:localhost/replica:0/task:0/device:CPU:0
gradients/Sum_grad/Shape_1: (Const): /job:localhost/replica:0/task:0/device:CPU:0
gradients/Sum_grad/range/start: (Const): /job:localhost/replica:0/task:0/device:CPU:0
gradients/Sum_grad/range/delta: (Const): /job:localhost/replica:0/task:0/device:CPU:0
gradients/Sum_grad/ones/Const: (Const): /job:localhost/replica:0/task:0/device:CPU:0
gradients/main_level/agent/main/online/network_1/ppo_head_0/policy_grad/Sum/reduction_indices: (Const): /job:localhost/replica:0/task:0/device:CPU:0
gradients/main_level/agent/main/online/network_1/ppo_head_0/strided_slice_grad/StridedSliceGrad

In [ ]:
df = pd.json_normalize(steps_data['steps'])
del steps_data
df['action_agree'] = np.where(df['openvino_action.action'] == df['tf_action.action'], 1, 0)
df['action_diff'] = np.abs(df['openvino_action.action'] - df['tf_action.action'])
action_analysis = df[['timestamp','seq','openvino_action.action','openvino_action.probability','tf_action.action', 'tf_action.probability', 'action_agree', 'action_diff']]
action_analysis.describe()

In [ ]:
# Creating histogram
fig, ax = plt.subplots(1, 3, figsize =(20, 5), sharey=True)
ax[0].hist(df['openvino_action.probability'], bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
ax[1].hist(df['tf_action.probability'], bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
ax[2].hist(df['tf_action_2.probability'], bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
plt.show()

In [ ]:
ov_a_c = pd.DataFrame(df['openvino_action.action'].value_counts(sort = False))
tf_a_c = pd.DataFrame(df['tf_action.action'].value_counts(sort = False))
a_c = ov_a_c.merge(tf_a_c, left_index=True, right_index=True).sort_index()
a_c.index.name='action'
a_c.rename(columns = {'tf_action.action':'tf', 'openvino_action.action': 'openvino'}, inplace = True)
display(a_c)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize =(15, 5), sharey=True)
fig.lab
ax[0].bar(a_c.index,a_c['openvino'][::-1])
plt.setp(ax[0], xlabel="OpenVINO")
plt.sca(ax[0])
plt.xticks(a_c.index,action_names[::-1],rotation='vertical')

ax[1].bar(a_c.index,a_c['tf'][::-1])
plt.setp(ax[1], xlabel="Tensorflow")
plt.sca(ax[1])
plt.xticks(a_c.index,action_names[::-1],rotation='vertical')
plt.show()